<a href="https://colab.research.google.com/github/suyash091/EEG-MULTIPLE-CHANNEL/blob/master/4_channel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!git clone https://github.com/tfjgeorge/ml-project.git

Cloning into 'ml-project'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 141 (delta 54), reused 141 (delta 54), pack-reused 0
Receiving objects: 100% (141/141), 4.42 MiB | 7.28 MiB/s, done.
Resolving deltas: 100% (54/54), done.


In [0]:
!mv '/content/ml-project' '/content/drive/My Drive'

In [0]:
from os.path import isfile
import zipfile
import numpy as np
import random

dataset_path = './data/EP.zip'

def get_dataset_file():
    if not isfile(dataset_path):
        import urllib
        origin = (
            'http://www.mindbigdata.com/opendb/MindBigData-MU-v1.0.zip'
        )
        print('Downloading data from %s' % origin)
        urlretrieve(origin, dataset_path)
    return open(dataset_path, 'rb')

def get_datasets():
    f = get_dataset_file()
    zf = zipfile.ZipFile(f)
        
    data = [ line for line in zf.open('MU.txt')]
    entire_dataset = []
    current_event = np.zeros(512 * 4 + 2)
    
    print('Reading data file')
    i = 0

    for l in data:
        ids, event, device, channel, code, size, data = l.decode("utf-8").split('\t')

        signals = np.array([int(val) for val in data.split(',')])
        
        current_event[1+ i*512:1+ i*512 + min(len(signals), 512)] = signals[:512]
        i += 1

        if i == 4: # we assume all channels from an event are in sequence
            current_event[-1] = int(code)
            current_event[0] = min(len(signals), 512)

            entire_dataset.append(current_event)
            current_event = np.zeros(512 * 4 + 2)
            i = 0

    random.seed(111) # deterministic
    random.shuffle(entire_dataset)

    entire_dataset = np.array(entire_dataset)
    return entire_dataset[:30000], entire_dataset[30000:]

def split_into_subsequences(data, n_sequences, length):
    output = np.zeros((data.shape[0]*n_sequences, length*14+1))
    for i in range(data.shape[0]):
        if n_sequences == 1:
            steps = 0
        else:
            steps = (data[i, 0] - length) / (n_sequences - 1)

        for j in range(n_sequences):
            output[i*n_sequences+j,:length]           = data[i, j*steps: j*steps+length]
            output[i*n_sequences+j,length:length*2]   = data[i, j*steps+512: j*steps+512+length]
            output[i*n_sequences+j,length*2:length*3] = data[i, j*steps+512*2: j*steps+512*2+length]
            output[i*n_sequences+j,length*3:length*4] = data[i, j*steps+512*3: j*steps+512*3+length]
            output[i*n_sequences+j,-1] = data[i, -1]

    return output

In [0]:

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from urllib.request import urlretrieve
#from sklearn.gaussian_process import GaussianProcess

In [0]:
(25/2050)*3586

43.73170731707317

In [15]:
train, test = get_datasets()
print(train.shape)
print(test.shape)

Reading data file
(30000, 2050)
(10983, 2050)


In [0]:
#train = train[:int(len(train)/3),:]
#test = test[:int(len(test)/3), :]

#train[train[:,-1] >= 0,-1] =  0
#test[test[:,-1] >= 0,-1] =  0

#train[:,-1] = train[:,-1] + 1
#test[:,-1] = test[:,-1] + 1

train_target = train[:,-1]
test_target = test[:,-1]

DECISIONBOUNDARY = 0

In [0]:
#FastFourierTransformation apply to the sets using the 4 sensors

train_f = np.zeros(train.shape)

for i in range(len(train_f)):
    length = train[i][0]
    train_f[i][0] = train[i][0]
    train_f[i][-1] = train[i][-1]
    
    for j in range(4):
        train_f[int(i)][int(1+512*j):int(1+512*j+length)] = np.abs(np.fft.fft(train[int(i)][int(1+512*j):int(1+512*j+length)]))
        
test_f = np.zeros(test.shape)

for i in range(len(test_f)):
    length = test[i,0]
    test_f[i,0] = test[i,0]
    test_f[i,-1] = test[i,-1]
    
    for j in range(4):
        test_f[i][int(1+512*j):int(1+512*j+length)] = np.abs(np.fft.fft(test[i][int(1+512*j):int(1+512*j+length)]))

In [0]:
#PCA with 25 components
pca = PCA(n_components = 250)
train_principal = pca.fit_transform(train_f)
test_principal = pca.transform(test_f)

In [20]:
test_principal.shape

(10983, 50)

In [23]:
#KNN
erreur = []
x = np.arange(1,20,1)
for n in (x):
    # train set
    neigh = KNeighborsClassifier(n_neighbors=n)
    neigh.fit(train_principal, train[:,-1])

    #test set
    total_correct = 0
    for i in range(test_principal.shape[0]):
        if neigh.predict(test_principal[i].reshape(1, -1))[0] == test[i,-1]:
            total_correct += 1

    erreur.append(float(total_correct) / test_principal.shape[0] * 100)
    print('Pourcentage correct %d:' % n, float(total_correct) / test_principal.shape[0] * 100)

Pourcentage correct 1: 34.30756623873258
Pourcentage correct 2: 34.43503596467268
Pourcentage correct 3: 34.389511062551215
Pourcentage correct 4: 34.20741145406537
Pourcentage correct 5: 34.489665847218426
Pourcentage correct 6: 34.55340071018848
Pourcentage correct 7: 34.36219612127834
Pourcentage correct 8: 34.45324592552126
Pourcentage correct 9: 34.50787580806701
Pourcentage correct 10: 34.56250569061277
Pourcentage correct 11: 34.55340071018848
Pourcentage correct 12: 34.480560866794136
Pourcentage correct 13: 34.25293635618684
Pourcentage correct 14: 34.09815168897387
Pourcentage correct 15: 34.34398616042976
Pourcentage correct 16: 34.398616042975505
Pourcentage correct 17: 34.389511062551215
Pourcentage correct 18: 34.54429572976418
Pourcentage correct 19: 34.49877082764272


In [0]:
#Import models from scikit learn module:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold   #For K-fold cross validation
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn import metrics
import numpy as np
from sklearn.metrics import roc_curve, auc, precision_score, confusion_matrix, explained_variance_score, max_error, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso

In [28]:

model = RandomForestClassifier(n_estimators=100)
model.fit(train_principal,train[:,-1])


predictions = model.predict(test_principal)
prc=precision_score(predictions,test[:,-1], average=None)
cfm=confusion_matrix(predictions,test[:,-1])
accuracy = metrics.accuracy_score(predictions,test[:,-1])
print(prc,cfm,accuracy)
#print(explained_variance_score(predictions,test))
#print(max_error(predictions,test))
#print(mean_absolute_error(predictions, test, multioutput='raw_values'))
#print(mean_squared_error(predictions, test, multioutput='raw_values'))

[1.         0.10314465 0.07177033 0.11417817 0.09452736 0.08168643
 0.10606061 0.0955335  0.09156627 0.09137709 0.07826087] [[2982   39   36   36   34   34   33   41   34   34   33]
 [   0   82   89   92   78   94   75   88  107   90  100]
 [   0   61   60   66   67   70   69   70   66   65   67]
 [   0  104   97   91   84   76   97   90   84   84   97]
 [   0   55   82   74   76   69   95   79   64   88   78]
 [   0   78   65   81   72   62   45   62   83   60   71]
 [   0   69   83   81   87   74   84   88   87   69   80]
 [   0   91   77   87   93   78   85   77   80   78   75]
 [   0   71   73   64   76   60   70   63   76   57   66]
 [   0   81   96   71   76   94   72   87   82   71   75]
 [   0   64   78   54   61   48   67   61   67   81   63]] 0.33906947100063733


In [30]:
model = LogisticRegression()
model.fit(train_principal,train[:,-1])


predictions = model.predict(test_principal)
prc=precision_score(predictions,test[:,-1], average=None)
cfm=confusion_matrix(predictions,test[:,-1])
accuracy = metrics.accuracy_score(predictions,test[:,-1])
print(prc,cfm,accuracy)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[1.         0.18238994 0.15789474 0.05771644 0.08457711 0.085639
 0.06313131 0.08436725 0.14819277 0.12612613 0.02981366] [[2982    1    0    1    0    0    0    0    0    0    0]
 [   0  145  129  123  123  115  126  141  128  121  120]
 [   0  125  132  129  136  116  133  117  121  128  137]
 [   0   56   50   46   61   44   53   44   60   71   41]
 [   0   65   63   81   68   69   67   60   82   69   76]
 [   0   57   63   68   54   65   58   64   63   59   63]
 [   0   69   92   59   59   57   50   82   75   51   66]
 [   0   69   60   64   53   70   71   68   79   57   71]
 [   0  117  109  108  130  109  116  121  123  102  117]
 [   0   70  104   94   95   87   92   79   71   98   90]
 [   0   21   34   24   25   27   26   30   28   21   24]] 0.34608030592734224


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [31]:
model = LinearRegression()
model.fit(train_principal,train[:,-1])


predictions = model.predict(test_principal)
#prc=precision_score(predictions,test[:,-1], average=None)
print(explained_variance_score(predictions,test[:,-1]))
print(max_error(predictions,test[:,-1]))
print(mean_absolute_error(predictions, test[:,-1], multioutput='raw_values'))
print(mean_squared_error(predictions, test[:,-1], multioutput='raw_values'))

-0.28568281533273665
39.534290805500724
[2.17300996]
[7.33731258]


In [32]:
model = RandomForestRegressor()
model.fit(train_principal,train[:,-1])


predictions = model.predict(test_principal)
print(explained_variance_score(predictions,test[:,-1]))
print(max_error(predictions,test[:,-1]))
print(mean_absolute_error(predictions, test[:,-1], multioutput='raw_values'))
print(mean_squared_error(predictions, test[:,-1], multioutput='raw_values'))

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


-0.08226809646697086
9.3
[1.95390148]
[6.88876354]


In [35]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import numpy as np

!pip install scipy
from scipy import signal

In [37]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1024, input_dim=250, activation='relu'))
model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(4096, activation='relu'))
model.add(tf.keras.layers.Dense(11, activation='softmax'))
alpha = 2e-3
batch_size = 256
num_epochs = 3000
model.compile(
      optimizer='Adam' ,
      loss=tf.keras.losses.sparse_categorical_crossentropy,
      metrics=['sparse_categorical_accuracy', 'accuracy']
  )

History = model.fit(
      x = train_principal,
      y = train[:,-1],
      batch_size = batch_size,
      epochs=num_epochs,
      #validation_data = (x_valid, y_valid),
      #callbacks = [checkPointer,tensorBoard]
  )
predictions = model.predict(test_principal)

30000/30000 [==============================] - 5s 170us/sample - loss: nan - sparse_categorical_accuracy: 0.0727 - acc: 0.0727
Epoch 67/3000
11520/30000 [==========>...................] - ETA: 3s - loss: nan - sparse_categorical_accuracy: 0.0696 - acc: 0.0696

KeyboardInterrupt: ignored